In [20]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import string
import re

In [3]:
#import bad-words.csv
df = pd.read_csv('C:\\Users\\HTY\\Desktop\\cs196\\bad-words.csv')
BWlist = df['BW'].values.tolist()

In [4]:
#set BasicFilter:
class BasicFilter():
    def __init__(self):
        self.keywords = set(BWlist)

    def filter(self,message,repl="*"):
        message = str(message).lower()
        for kw in self.keywords:
            message = message.replace(kw, repl)
        return message

In [5]:
#try basic filter
print(BasicFilter().filter('fuck u'))

*k u


In [32]:
#import train data
train = pd.read_csv('C:\\Users\\HTY\\Desktop\\cs196\\train.csv')
test = pd.read_csv('C:\\Users\\HTY\\Desktop\\cs196\\test.csv')
sample = pd.read_csv('C:\\Users\\HTY\\Desktop\\cs196\\sample_submission.csv')
tl = pd.read_csv('C:\\Users\\HTY\\Desktop\\cs196\\test_labels.csv')

In [41]:
test.head()

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.


In [40]:
#data cleaning
COMMENT = 'comment_text'
train[COMMENT].fillna("unknown", inplace=True)
label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

In [45]:
#develop NB-SVM filter

def NBfilter(stri):
    d = []
    d.append({'id':'i','toxic':'','severe_toxic':'','obscene':'','threat':'','insult':'','identity_hate':''})
    subm = pd.DataFrame(d)
    e = []
    e.append({'id':'i','comment_text':stri})
    test = pd.DataFrame(e)
    return eva(test,subm)

def tokenize(s):
    return re.compile(f'([{string.punctuation}“”¨«»®´·º½¾¿¡§£₤‘’])').sub(r' \1 ', s).split()

def pr(x,y_i, y):
    p = x[y==y_i].sum(0)
    return (p+1) / ((y==y_i).sum()+1)
    
def get_mdl(x,y):
    y = y.values
    r = np.log(pr(x,1,y) / pr(x,0,y))
    m = LogisticRegression(C=4, dual=True)
    x_nb = x.multiply(r)
    return m.fit(x_nb, y), r

def eva(test,subm):
    #n = train.shape[0]
    vec = TfidfVectorizer(ngram_range=(1,2), tokenizer=tokenize,
            min_df=3, max_df=0.9, strip_accents='unicode', use_idf=1,
            smooth_idf=1, sublinear_tf=1 )
    trn_term_doc = vec.fit_transform(train[COMMENT])
    test_term_doc = vec.transform(test[COMMENT])
    x = trn_term_doc
    test_x = test_term_doc
    preds = np.zeros((len(test), len(label_cols)))
    for i, j in enumerate(label_cols):
        m,r = get_mdl(x,train[j])
        preds[:,i] = m.predict_proba(test_x.multiply(r))[:,1]
    submid = pd.DataFrame({'id': subm['id']})
    return pd.concat([submid, pd.DataFrame(preds, columns = label_cols)], axis=1)

In [47]:
toReturn = NBfilter('fuck u')

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [48]:
toReturn

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,i,1.0,0.781594,1.0,0.001251,0.991038,0.01853
